
# Cleaning & Geolocating

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [3]:
df = pd.read_excel('raw_data/2023.xlsx')
df.to_csv('raw_data/2023.csv', index=False)
pd.set_option('display.max_colwidth', None)
df.sample(10)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments
48293,2023-12-16 13:33:00,ES260,956202,ESC: ES260 - 475 - 96 St - Outside Area,This escalator is out of service for Liftnet Event : Safety Brake Release #3,BRP,Closed,Safety Brake Release #3,REPAIR,Repair,2023-12-16 22:00:00,2023-12-16 16:55:00,ST231216.txt - 1592,External Monitoring System,EE-LNOUT,2023-12-16 13:35:00,CL,NaN
14765,2023-04-20 03:33:00,EL729X,834746,ELE: EL729X - 412 - Fulton St - Brooklyn Bound Plat,NaN,TPE,Closed,NaN,REPAIR,Repair,2023-04-22 12:00:00,2023-04-20 09:39:00,NaN,Phone,EE-MANOUT,2023-04-20 03:33:00,CL,**This elevator is out of service as reported by S/S Dasgupta***\n**This machine is out of service for required repairs. NYC Transit is not responsible for the maintenance or repair of this unit** @ 09:19 Notified Supt. Bertolino to check elevator status. As per E&EM Rapoport this elevator is in service.
36577,2023-09-20 07:00:00,ES624,922050,ESC: ES624 - 471 - 34 St-Hudson Yards - Mezz A,This escalator is out of service for preventive maintenance,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-09-20 15:00:00,2023-09-20 12:40:00,NaN,Phone,EE-MANOUT,2023-09-20 07:14:00,CL,"E&EM Pharous & Schoendorf and E&EH Ahmed arrive at 0530. Here for schedule ""D"" PM. Machine out of service at 0700. WO#7750996 E&EH Ahmed RTS after completing PM @1240 hrs."
22166,2023-06-10 21:25:00,ES328,881304,ESC: ES328 - 233 - Delancey St-Essex St - Queens Bound Plat,NaN,VAN,Closed,Ready,UNDERINVESTG,Under Investigation,2023-06-11 06:00:00,2023-06-11 01:00:00,ST230610.txt - 2804,External Monitoring System,EE-LNOUT,2023-06-10 21:38:00,CL,NaN
58,2023-01-01 16:48:00,EL104,791369,ELE: EL104 - 300 - 191 St,NaN,BRP,Closed,Door Opening/ R. Door Open Limit failure,REPAIR,Repair,2023-01-02 01:00:00,2023-01-02 00:15:00,ST230101.txt - 1297,External Monitoring System,EE-LNOUT,2023-01-01 16:54:00,CL,NaN
23592,2023-06-20 08:13:00,EL235,885302,ELE: EL235 - 225 -47-50 Sts-Rockefeller Ctr - Queens Bound Plat,NaN,FR,Cancelled,DOWN SLOWDOWN LIMIT OPEN,UNDERINVESTG,Under Investigation,2023-06-20 17:00:00,NaT,ST230620.txt - 1180,External Monitoring System,EE-LNOUT,2023-06-20 08:23:00,C,As per Liftnet Galaxy Client machine is running.
10956,2023-03-23 11:04:00,ES214,823792,ESC: ES214 - 400 - 59 St - Brooklyn Bound Local Plat,NaN,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-03-23 16:00:00,2023-03-23 12:55:00,NaN,Phone,EE-MANOUT,2023-03-23 11:04:00,CL,NaN
17584,2023-05-09 19:40:00,EL222,851905,ELE: EL222 - 166 - 14 St - Brooklyn Bound Plat,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-05-10 07:00:00,2023-05-10 04:30:00,NaN,Phone,EE-MANOUT,2023-05-09 19:41:00,CL,*** This elevator is out of service due to Elevator Quarterly Maintenance*** WO#6365537\nE&EM George and E&EH Dowers arrived for Quarterly PM. taking machine out of service at 19:40 5/10/23 @ 0430 (Rosario) E&EM George and E&EH Dowers reports completed\n quarterly maintenance. RTS\n\n
9850,2023-03-17 06:45:00,EL144,820735,ELE: EL144 - 153 - 125 St,NaN,POW,Closed,Power Fault,UNDERINVESTG,Under Investigation,2023-03-17 15:00:00,2023-03-17 11:00:00,ST230317.txt - 707,External Monitoring System,EE-LNOUT,2023-03-17 06:55:00,CL,NaN
2069,2023-01-16 18:48:00,ES423,797168,ESC: ES423 - 221 - 21 St-Queensbridge - Brooklyn Bound Plat,NaN,VAN,Closed,Ready,UNDERINVESTG,Under Investigation,2023-01-17 03:00:00,2023-01-16 19:45:00,ST230116.txt - 1899,External Monitoring System,EE-LNOUT,2023-01-16 18:57:00,CL,NaN


In [4]:
%%R
df <- read_csv('raw_data/2023.csv')

Rows: 50296 Columns: 18
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (13): Common Name, Equipment Description, Executive Comment, Outage Cod...
dbl   (1): Outage
dttm  (4): Out of Service Date, Estimated Return to Service Date, Actual Ret...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

In [5]:
# filter down to elevators only
df_ele = df[df['Equipment Description'].str.startswith('ELE')]
df_ele.sample(10)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments
12268,2023-04-02 06:49:00,EL204,827928,ELE: EL204 - 402 - Grand Central-42 St - Outside Area,NaN,FR,Cancelled,24V +A Fuse (FU12),UNDERINVESTG,Under Investigation,2023-04-02 15:00:00,NaT,ST230402.txt - 419,External Monitoring System,EE-LNOUT,2023-04-02 06:58:00,C,Both Liftnets are showing conflicting information. Spoke to MS1 Sookdhan. Spoke to S/A Jones #44620 elevator is working as design.
8112,2023-03-04 00:55:00,EL306,815864,ELE: EL306 - 338 - Atlantic Av-Barclays Ctr - Manhattan Bound Plat,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-03-04 09:00:00,2023-03-04 05:30:00,NaN,Phone,EE-MANOUT,2023-03-04 00:56:00,CL,"***This elevator is out of service for Monthly PM***\nWO5898910 EAM OUT815864\n3/04/23 0055hrs {Jackson,R}E&EM Figueroa, E&EH Lattanzio, and E&EH Stewart taking unit OOS for Monthly PM Maintenance completed. Machine BIS @05:30 hrs"
21035,2023-06-02 01:38:00,EL133,878378,ELE: EL133 - 219 - 161 St-Yankee Stadium,NaN,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-06-02 09:00:00,2023-06-02 02:55:00,NaN,Phone,EE-MANOUT,2023-06-02 01:38:00,CL,NaN
40379,2023-10-19 00:20:00,EL498,933801,ELE: EL498 - 209 - Far Rockaway-Mott Av - Island Plat,This Elevator is out of service for Quarterly Maintenance.,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-10-19 09:00:00,2023-10-19 05:00:00,NaN,Phone,EE-MANOUT,2023-10-19 00:23:00,CL,E&EM Grant and E&EH Taiwo arrived at El498 at 0010 hours for quarterly PM. Out of service 0015 hours. WO#8021129 PM Complete RTS @0500HRS
5242,2023-02-09 13:15:00,EL135,807345,ELE: EL135 - 219 - 161 St-Yankee Stadium,NaN,APC,Closed,NaN,PLANNEDWORK,Planned Work,2023-02-09 22:00:00,2023-02-09 14:20:00,NaN,Phone,EE-MANOUT,2023-02-09 13:17:00,CL,***This elevator is out of service for A&P for CBA Contractor to perform the survey***\n\n\n\nE&EM Lazariev taking unit OOS to provide A&P for CBA Contractor to perform the survey. WO#5683516 E&EM Garcia: Completed survey for contractors tested machine run RTS at 1420H
40738,2023-10-22 03:30:00,EL182,934924,ELE: EL182 - 421 - Gun Hill Rd,This elevator is out of service for Glass Cleaning,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-10-22 11:00:00,2023-10-22 04:30:00,NaN,Phone,EE-MANOUT,2023-10-22 03:33:00,CL,E&EM Louis and E&EH Titus arrived at el182 for glass cleaning. WO8264718 Completed glass cleaning. Cleaned in car glass panels. Test run and BIS at 0430 hours
27518,2023-07-17 14:01:00,EL328,896284,ELE: EL328 - 230 - B'way-Lafayette St - Outside Area,NaN,BRP,Closed,Fire Phase 1,REPAIR,Repair,2023-07-17 23:00:00,2023-07-17 17:20:00,ST230717.txt - 2115,External Monitoring System,EE-LNOUT,2023-07-17 14:16:00,CL,NaN
13935,2023-04-14 07:17:00,EL458,832497,ELE: EL458 - 965 - FREIGHT ELEVATOR AT WOODSIDE EMD,NaN,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-04-14 16:00:00,2023-04-14 14:30:00,NaN,Phone,EE-MANOUT,2023-04-14 07:19:00,CL,***This elevator is out of service for top landing hoist way door hanger rollers and gibs replacement***\n\n\n\nRichmond tech. Soto/Moran taking elevator OOS to replace top landing hoist way door hanger rollers and gibs. WO#6214647 Richmond tech. Soto/Moran: Cleaned out Removed old hoist-way door hanger rollers at top landing \nWe are in the process of installing new hanger rollers at hoist-way door top landing \n\n Richmond tech. Soto/Moran: Cleaned out Removed old hoist-way door hanger rollers top landings We are in the process to install new hanger rollers hoist-way door top landing.\n\n Richmond tech. Moran email: Finished installed hanger rollers hoistway door top landing. Adjusted door for proper clearance. RTS @14:30 \n\n
6383,2023-02-18 00:26:00,EL435,810574,ELE: EL435 - 259 - Kew Gardens-Union Tpke - Queens Bound Plat,NaN,BRP,Closed,Fir

In [6]:
# create column `station MRN` to merge with `Station ID` to geolocate
df_ele['Station MRN'] = df_ele['Equipment Description'].str.extract(r'ELE: EL\d{2,3}\w? - (\d{2,3})\s-')

In [8]:
# check what the are the missing station MRN
# missing values are all elevators that are either located in New Utrecht Av (MRN: 73) or Dyckman St Quarters(MRN: TBD)

print(df_ele[df_ele['Station MRN'].isna()].shape)
df_ele[df_ele['Station MRN'].isna()].to_csv('missing_data/2023_missing_station_mrn.csv', index=False)
df_ele[df_ele['Station MRN'].isna()].sample(10)

(147, 19)


,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN
31990,2023-08-18 00:13:00,EL740,908770,ELE: EL740 - 73 - New Utrecht - Location TBD,NaN,INS,Closed,NaN,INSPECTION,Inspection,2023-08-18 08:00:00,2023-08-18 04:15:00,NaN,Phone,EE-MANOUT,2023-08-18 00:13:00,CL,"***This elevator is out of service for ASME CAT 1 Inspection***\nWO6596758 EAM OUT908770\n8/18/23 0013hrs {Jackson,R}E&EM's Daly and Walsh taking unit OOS for CAT 1 Inspection with Third party Inspector Camacho on site EL740 passed cat 1 inspection. Returned to service at 04:15 am",NaN
7071,2023-02-23 13:19:00,EL741,812858,ELE: EL741 - 73 - New Utrecht - Location TBD,NaN,BRP,Closed,Front Gate Switch Bypass Switch Failure,UNDERINVESTG,Under Investigation,2023-02-23 22:00:00,2023-02-23 17:30:00,ST230223.txt - 2138,External Monitoring System,EE-LNOUT,2023-02-23 13:27:00,CL,NaN,NaN
44792,2023-11-21 00:10:00,EL741,946059,ELE: EL741 - 73 - New Utrecht - Location TBD,****This Elevator is Out for Monthly Preventative Maintenance**** \n\nWO8275719,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-11-21 06:00:00,2023-11-21 04:30:00,NaN,Phone,EE-MANOUT,2023-11-21 00:12:00,CL,E&EM Furmanov and E&EH Xie Liang did successfully Monthly maintenance El741 . PM completed . BIS at 04:30 E&EM Furmanov and E&EH Xie Liang arrived at 2330 for Monthly PM,NaN
20984,2023-06-01 17:09:00,EL740,878254,ELE: EL740 - 73 - New Utrecht - Location TBD,NaN,BRP,Closed,Positioning Signal HT1/HT2 Fault,UNDERINVESTG,Under Investigation,2023-06-02 02:00:00,2023-06-01 19:30:00,ST230601.txt - 4291,External Monitoring System,EE-LNOUT,2023-06-01 17:18:00,CL,NaN,NaN
41266,2023-10-26 07:35:00,EL156,936422,ELE: EL156 - TBD - Dyckman St Quarters - Location TBD,This elevator is out of service to inspect guide shoes and rails,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-10-26 15:00:00,2023-10-26 12:00:00,NaN,Phone,EE-MANOUT,2023-10-26 07:37:00,CL,NaN,NaN
34352,2023-09-05 12:10:00,EL741,915449,ELE: EL741 - 73 - New Utrecht - Location TBD,This elevator is out of service for Verification of the As-built drawings.,APC,Closed,NaN,PLANNEDWORK,Planned Work,2023-09-05 21:00:00,2023-09-05 12:20:00,NaN,Phone,EE-MANOUT,2023-09-05 12:11:00,CL,NaN,NaN
35355,2023-09-12 12:56:00,EL739,918347,ELE: EL739 - 73 - New Utrecht - Location TBD,S/A Cheng #147466 reported the Street landing hall-call button is not taking calls.,BRP,Closed,NaN,REPAIR,Repair,2023-09-12 21:00:00,2023-09-12 17:30:00,NaN,Phone,EE-MANOUT,2023-09-12 12:57:00,CL,NaN,NaN
29807,2023-08-02 07:55:00,EL739,902302,ELE: EL739 - 73 - New Utrecht - Location TBD,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-08-02 16:00:00,2023-08-02 12:40:00,NaN,Phone,EE-MANOUT,2023-08-02 07:57:00,CL,***This elevator is out of service for preventive maintenance***\n\n\n\nE&EM Samuel & E&EH Shiva arrived at 0715hrs. and taking unit OOS @07:55hrs. for monthly PM. WO#7501791. E&EM Samuel & E&EH Shiva completed monthly PM. RTS @12:40hrs.,NaN
22530,2023-06-13 10:15:00,EL156,882314,ELE: EL156 - TBD - Dyckman St Quarters - Location TBD,NaN,INS,Closed,NaN,INSPECTION,Inspection,2023-06-13 19:00:00,2023-06-13 12:45:00,NaN,Phone,EE-MANOUT,2023-06-13 10:18:00,CL,"***This elevator is out of service for CAT1 Inspection*** \n\n\nE&EM Stewart, E&EM Rosario, and E&EH Tenecela arrived at 1010hrs. Here for CAT 1 inspection. OOS@1015hrs. wo#6121520 E&EM Stewart, E&EM Rosario, and E&EH Tenecela completed and passed CAT 1 inspection. BIS @1245hrs.",NaN
37326,2023-09-26 08:45:00,EL740,924804,ELE: EL740 - 73 - New Utrecht - Location TBD,E&E personnel removed this unit from service to correct defects.,MRP,Closed,NaN,PLANNEDWORK,Planned Work,2023-09-26 15:00:00,2023-09-26 10:30:00,NaN,Phone,EE-MANOUT,2023-09-26 08:45:00,CL,NaN,NaN


In [10]:
# geocoding my dataset: merging stations by `Station ID` and `Station MRN` (my dataset)

# convert string to float
df_ele['Station MRN'] = df_ele['Station MRN'].astype('Int64')

# stations with MRN > 523 are not regular subway stations and not geolocatable
# drop all stations MRN > 523 and save to a new file
df_ele[df_ele['Station MRN'] <= 523].to_csv('processed_data/2023_ele.csv', index=False)
df_ele[df_ele['Station MRN'] > 523].to_csv('missing_data/2023_ele_not_subway.csv', index=False)

# filter down to geolocatable columns
df_ele = df_ele[df_ele['Station MRN'] <= 523]
df_ele.sample(5)

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN
42631,2023-11-05 17:23:00,EL502,940020,ELE: EL502 - 211 - Bedford Pk Blvd - Manhattan Bound Platform,This Elevator is out of service due to LIFTNET event: Trouble,VAN,Closed,Trouble,REPAIR,Repair,2023-11-06 02:00:00,2023-11-05 20:15:00,ST231105.txt - 1987,External Monitoring System,EE-LNOUT,2023-11-05 17:36:00,CL,NaN,211
36072,2023-09-16 20:26:00,EL429,920484,ELE: EL429 - 273 - Queens Plaza - Manhattan Bound Plat,This Elevator is out service reported by S/A Ahmed pass# 005768,VAN,Closed,NaN,UNDERINVESTG,Under Investigation,2023-09-17 08:00:00,2023-09-16 21:00:00,NaN,Phone,EE-MANOUT,2023-09-16 20:26:00,CL,NaN,273
41928,2023-10-31 23:00:00,EL719,938016,ELE: EL719 - 106 - Fulton St - Upper Plat,This Elevator is out of service for Inspection,INS,Closed,NaN,INSPECTION,Inspection,2023-11-01 07:00:00,2023-11-01 05:00:00,NaN,Phone,EE-MANOUT,2023-10-31 23:09:00,CL,EEM Daly and EEM Hill arrived at el719 at (22:50) and are removing from service el719 at (23:00) for cat 1 and cat 5 inspection. WO #7751735 El719 passed cat 1 and cat 5 inspection. Returned el719 back to service at 05:00,106
31100,2023-08-11 09:06:00,EL219,906233,ELE: EL219 - 015 - 14 St-Union Sq - Queens Bound Plat,NaN,EUF,Closed,NaN,REPAIR,Repair,2023-08-12 18:00:00,2023-08-11 19:00:00,NaN,Phone,EE-MANOUT,2023-08-11 09:18:00,CL,NaN,15
38299,2023-10-03 16:11:00,EL227,927835,ELE: EL227 - 164 - 34 St-Penn Station - Island Plat,NaN,FR,Cancelled,Stop Switch,UNDERINVESTG,Under Investigation,2023-10-04 01:00:00,NaT,ST231003.txt - 3533,External Monitoring System,EE-LNOUT,2023-10-03 16:29:00,C,"As per MS1 Lin, machine is running.",164


In [11]:
# load the geostation file
df_geostation = pd.read_csv('raw_data/subway_stations.csv')

# merge the geostation file with the df_ele
df_ele_merged = df_ele.merge(df_geostation, left_on='Station MRN', right_on='Station ID')

pd.set_option('display.max_columns', 500)

# rename the columns
df_ele_merged = df_ele_merged.rename(columns={'GTFS Latitude': 'lat',  'GTFS Longitude': 'long'})


In [12]:
# check the merged file 

# save the merged file
df_ele_merged.to_csv('processed_data/2023_ele_geocode.csv', index=False)
print(df_ele_merged.shape)
df_ele_merged.sample(5)


# WHY IS THERE MORE ROWS THAN THE ORIGINAL FILE??

(18208, 35)


,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference
14444,2023-10-17 12:23:00,EL719,932994,ELE: EL719 - 106 - Fulton St - Upper Plat,"This Elevator is out service reported by Vasquez, Edward",BRP,Closed,NaN,REPAIR,Repair,2023-10-17 21:00:00,2023-10-17 13:15:00,NaN,Phone,EE-MANOUT,2023-10-17 12:26:00,CL,NaN,106,106,628,M22,BMT,Jamaica,Fulton St,M,J Z,Subway,40.710374,-74.007582,Brooklyn,Broad St,1,NaN,POINT (-74.007582 40.710374)
1174,2023-01-24 03:39:00,EL441,800456,ELE: EL441 - 223 - Lexington Av/63 St,NaN,UNI,Cancelled,Fire Phase 1,UNDERINVESTG,Under Investigation,2023-01-24 12:00:00,NaT,ST230124.txt - 778,External Monitoring System,EE-LNOUT,2023-01-24 03:51:00,C,Machine was on fire service due to entrapment on EL442,223,223,223,B08,IND,63rd St,Lexington Av/63 St,M,F Q,Subway,40.764629,-73.966113,Uptown - Queens,Downtown & Brooklyn,1,NaN,POINT (-73.966113 40.764629)
3192,2023-03-07 23:41:00,EL601,817144,ELE: EL601 - 471 - 34 St-Hudson Yards - Mezz A,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-03-08 08:00:00,2023-03-08 00:00:00,NaN,Phone,EE-MANOUT,2023-03-07 23:42:00,CL,"***This elevator is out of service for Acces & Protection for Safety***\nEAM OUT817144\n3/07/23 2341hrs {Jackson,R}E&EM Dean and E&EH Ahmed taking unit OOS for A&P for Safety while working on el602 finish fireman service unit BIS @ 0000hrs",471,471,471,726,IRT,Flushing,34 St-Hudson Yards,M,7,Subway,40.755882,-74.001910,Queens,NaN,1,NaN,POINT (-74.00191 40.755882)
9640,2023-07-10 09:50:00,EL341,893159,ELE: EL341 - 188 - Euclid Av - Outside Area,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-07-10 18:00:00,2023-07-10 10:15:00,NaN,Phone,EE-MANOUT,2023-07-10 09:52:00,CL,***This Escalator is out for A&P for EMD Inspection***\n\n\n07/10/23 E&EM Liu47 arrived@0830hrs. A&P for EMD.\n\nE&EM Rodriguez arrived at EL341. Here to provide A&P for EMD\n\nWO#6548153 E&EM Rodriguez report: EMD repaired 1 smoke detector in the pit. RTS @10:15hrs.,188,188,188,A55,IND,8th Av - Fulton St,Euclid Av,Bk,A C,Subway,40.675377,-73.872106,Manhattan,Lefferts - Rockaways,1,NaN,POINT (-73.872106 40.675377)
7102,2023-05-23 16:47:00,EL127,874540,ELE: EL127 - 430 - Simpson St,NaN,VAN,Closed,Main Line Disconnect,UNDERINVESTG,Under Investigation,2023-05-24 01:00:00,2023-05-23 19:10:00,ST230523.txt - 3200,External Monitoring System,EE-LNOUT,2023-05-23 16:56:00,CL,NaN,430,430,430,217,IRT,Lenox - White Plains Rd,Simpson St,Bx,2 5,Elevated,40.824073,-73.893064,Wakefield - Eastchester,Manhattan,1,NaN,POINT (-73.893064 40.824073)


## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [13]:
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm

import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df_ele_merged['lat']
        longitudes = df_ele_merged['long']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df_ele_merged)))

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df_ele_merged['lat'], df_ele_merged['long']) 
census_geos_df.head()

  0%|          | 0/18208 [00:00<?, ?it/s]

,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,INTPTLAT,FUNCSTAT,NAME,OBJECTID,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,610,360470296002002,+40.6008553,2002,0,36,2002,210701004650915,BK,+40.6008553,S,Block 2002,4615836,029600,-073.9943207,2,20213,242,-073.9943207,G5040,L,U,047
1,,0,360050403021003,+40.8669415,1003,0,36,1003,210701006017414,BK,+40.8669415,S,Block 1003,1475683,040302,-073.8935996,1,3799,0,-073.8935996,G5040,L,U,005
2,,593,360610034004000,+40.7305461,4000,0,36,4000,210701008621623,BK,+40.7305461,S,Block 4000,2844262,003400,-073.9817046,4,19330,415,-073.9817046,G5040,L,U,061
3,,0,360810208001000,+40.6998869,1000,0,36,1000,210701006104178,BK,+40.6998869,S,Block 1000,1848413,020800,-073.8089158,1,13876,0,-073.8089158,G5040,L,U,081
4,,0,360810033011019,+40.7492525,1019,0,36,1019,210701006117049,BK,+40.7492525,S,Block 1019,1404062,003301,-073.9374018,1,3969,0,-073.9374018,G5040,L,U,081


In [14]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,360470296002002,36,047,029600,2002
1,360050403021003,36,005,040302,1003
2,360610034004000,36,061,003400,4000
3,360810208001000,36,081,020800,1000
4,360810033011019,36,081,003301,1019
...,...,...,...,...,...
18203,360610137003000,36,061,013700,3000
18204,360610137003000,36,061,013700,3000
18205,360610137003000,36,061,013700,3000
18206,360050224031001,36,005,022403,1001


## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [16]:
df_with_geos = pd.concat(
    [ 
        df_ele_merged.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,Out of Service Date,Common Name,Outage,Equipment Description,Executive Comment,Outage Code,Status,External Source Note,Reason Shown to Public,Reason Shown to Public Description,Estimated Return to Service Date,Actual Return to Service Date,Reference,Source,Service Code,Date Created,Status Code,Outage Comments,Station MRN,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,lat,long,North Direction Label,South Direction Label,ADA,ADA Notes,Georeference,GEOID,STATE,COUNTY,TRACT,BLOCK
0,2023-01-01 00:14:00,EL376,791238,ELE: EL376 - 068 - Bay Pkwy - Outside Area,NaN,PM,Closed,NaN,MAINTENANCE,Maintenance,2023-01-01 07:00:00,2023-01-01 05:30:00,NaN,Phone,EE-MANOUT,2023-01-01 00:14:00,CL,***This Elevator is out of service for Maintenance***\n\n\nE&EM R.Brown and E&Eh K.Singh arrived for maintenance at 00:05. WO #5407916 PM Completed and RTS @5.30,68,68,68,B21,BMT,West End,Bay Pkwy,Bk,D,Elevated,40.601875,-73.993728,Manhattan,Coney Island,1,NaN,POINT (-73.993728 40.601875),360470296002002,36,047,029600,2002
1,2023-01-01 01:04:00,EL189,791257,ELE: EL189 - 212 - Kingsbridge Rd - Outside Area,NaN,AP,Closed,NaN,PLANNEDWORK,Planned Work,2023-01-01 09:00:00,2023-01-01 01:18:00,NaN,Phone,EE-MANOUT,2023-01-01 01:05:00,CL,\n\n***This elevator is out of service for Access & Protection for Safety***,212,212,212,D04,IND,Concourse,Kingsbridge Rd,Bx,B D,Subway,40.866978,-73.893509,Bedford Pk Blvd & 205 St,Manhattan,1,NaN,POINT (-73.893509 40.866978),360050403021003,36,005,040302,1003
2,2023-01-01 02:01:00,EL293,791274,ELE: EL293 - 119 - 1 Av - Brooklyn Bound Platform,NaN,EUF,Closed,UPS Battery Failure,REPAIR,Repair,2023-01-01 11:00:00,2023-01-01 04:50:00,ST230101.txt - 192,External Monitoring System,EE-LNOUT,2023-01-01 02:08:00,CL,NaN,119,119,119,L06,BMT,Canarsie,1 Av,M,L,Subway,40.730953,-73.981628,8 Av,Brooklyn,1,NaN,POINT (-73.981628 40.730953),360610034004000,36,061,003400,4000
3,2023-01-01 02:08:00,EL449X,791275,ELE: EL449X - 279 - Sutphin Blvd-Archer Av - Mezz,NaN,TPE,Closed,NaN,REPAIR,Repair,2023-01-05 14:00:00,2023-01-05 03:06:00,NaN,Phone,EE-MANOUT,2023-01-01 02:09:00,CL,"***This elevator is out of service as reported by SS Patel010647***\n**This machine is out of service for required repairs. NYC Transit is not responsible for the maintenance or repair of this unit**\n At 22:45 on 01/02/2023 MOW#85 states customers entrapped in EL449x at Sutphin blvd. No medical requested. Notified MS1 Pukhalski MS1 Pukhalskiy updated by email Checked the EL449X, nobody inside. Sent email to Dist. Field Office for an update. ERTS changed to 01/05/23.\n\n\n S/S Patel reports found machine running while on station. RTS",279,279,279,G06,IND,Queens - Archer,Sutphin Blvd-Archer Av-JFK Airport,Q,E J Z,Subway,40.700486,-73.807969,Jamaica Center,Manhattan,1,NaN,POINT (-73.807969 40.700486),360810208001000,36,081,020800,1000
4,2023-01-01 05:43:00,EL428,791299,ELE: EL428 - 273 - Queens Plaza - Outside Area,NaN,VAN,Closed,NaN,REPAIR,Repair,2023-01-01 13:00:00,2023-01-01 09:10:00,NaN,Phone,EE-MANOUT,2023-01-01 05:44:00,CL,NaN,273,273,273,G21,IND,Queens Blvd,Queens Plaza,Q,E M R,Subway,40.748973,-73.937243,Forest Hills - Jamaica,Manhattan,1,NaN,POINT (-73.937243 40.748973),360810033011019,36,081,003301,1019


In [17]:
df_with_geos.to_csv('data/2023_subway_censusgeo.csv', index=False)